In [2]:
%run ../Requirements.ipynb

24/06/20 16:13:56 WARN Utils: Your hostname, Ronits-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.7 instead (on interface en0)
24/06/20 16:13:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/20 16:13:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
conn = mysql.connector.connect(
    host='127.0.0.1',
    user='root',
    password='ronitgupta28',
    database='MusicFusion'
)
cursor = conn.cursor()

Combined Music Artist Details

In [14]:
df = spark.sql(
    """
    SELECT DISTINCT SM.Artists
    FROM PARQUET.`/Users/ronitguptaaa/Documents/Music-Fusion-APP/Staging/AppleMusic` AS AM
    INNER JOIN PARQUET.`/Users/ronitguptaaa/Documents/Music-Fusion-APP/Staging/SpotifyMusic` AS SM 
    ON LTRIM(RTRIM(LOWER(AM.`Artist Name`))) = LTRIM(RTRIM(LOWER(SM.Artists)))
    LEFT JOIN PARQUET.`/Users/ronitguptaaa/Documents/Music-Fusion-APP/Staging/YoutubeMusic` AS YM
    ON LTRIM(RTRIM(LOWER(AM.`Artist Name`))) = LTRIM(RTRIM(LOWER(YM.`Artist Name`)))
    """
)

In [15]:
artists_df = df.select(F.col('Artists')).distinct()
artists_list = [row['Artists'] for row in artists_df.collect()]

news_api_key = News_API

news_data = []
for artist in artists_list:
    url = f"https://newsapi.org/v2/everything?q={artist}&pageSize=10&apiKey={news_api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        articles = response.json()['articles']
        for article in articles:
            news_data.append({
                'Artist': artist,
                'Title': article.get('title'),
                'Description': article.get('description'),
                'URL': article.get('url'),
                'Content': article.get('content')
            })
    else:
        print(f"Failed to fetch news for artist: {artist}, Status Code: {response.status_code}")

news_df = pd.DataFrame(news_data)
news_spark_df = spark.createDataFrame(news_df)

news_spark_df.write.mode('overwrite').format('parquet').save('../Mart/MusicFusion-Details')

24/06/20 15:55:13 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [22]:
df = pd.read_parquet('/Users/ronitguptaaa/Documents/Music-Fusion-APP/Mart/MusicFusion-Details')
df = df.dropna()

In [24]:
columns = ", ".join([f"`{col}` VARCHAR(1024)" for col in df.columns])
drop_table_query = f"DROP TABLE IF EXISTS MusicFusionDetails;"
cursor.execute(drop_table_query)
create_table_query = f"CREATE TABLE IF NOT EXISTS MusicFusionDetails ({columns});"
cursor.execute(create_table_query)

for index, row in df.iterrows():
    values = tuple(row.values)
    insert_query = f"INSERT INTO MusicFusionDetails VALUES {values};"
    cursor.execute(insert_query)
    
alter_table_query = f"DELETE FROM MusicFusionDetails WHERE Title = '[Removed]'"
cursor.execute(alter_table_query)

conn.commit()


Apple Music Artist Details

In [25]:
df = spark.sql("SELECT DISTINCT * FROM PARQUET.`/Users/ronitguptaaa/Documents/Music-Fusion-APP/Staging/AppleMusic`")

In [26]:
artists_df = df.select(F.col('Artist Name')).distinct()
artists_list = [row['Artist Name'] for row in artists_df.collect()]

news_api_key = News_API

news_data = []
for artist in artists_list:
    url = f"https://newsapi.org/v2/everything?q={artist}&pageSize=10&apiKey={news_api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        articles = response.json()['articles']
        for article in articles:
            news_data.append({
                'Artist': artist,
                'Title': article.get('title'),
                'Description': article.get('description'),
                'URL': article.get('url'),
                'Content': article.get('content')
            })
    else:
        print(f"Failed to fetch news for artist: {artist}, Status Code: {response.status_code}")

news_df = pd.DataFrame(news_data)
news_spark_df = spark.createDataFrame(news_df)

news_spark_df.write.mode('overwrite').format('parquet').save('../Mart/AppleMusic-Details')

24/06/20 16:06:37 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [28]:
df = pd.read_parquet('/Users/ronitguptaaa/Documents/Music-Fusion-APP/Mart/AppleMusic-Details')
df = df.dropna()

In [29]:
columns = ", ".join([f"`{col}` VARCHAR(1024)" for col in df.columns])
drop_table_query = f"DROP TABLE IF EXISTS AppleMusicDetails;"
cursor.execute(drop_table_query)
create_table_query = f"CREATE TABLE IF NOT EXISTS AppleMusicDetails ({columns});"
cursor.execute(create_table_query)

for index, row in df.iterrows():
    values = tuple(row.values)
    insert_query = f"REPLACE INTO AppleMusicDetails VALUES {values};"
    cursor.execute(insert_query)

alter_table_query = f"DELETE FROM AppleMusicDetails WHERE Title = '[Removed]'"
cursor.execute(alter_table_query)
conn.commit()

Spotify Artist Details

In [30]:
df = spark.sql("SELECT DISTINCT * FROM PARQUET.`/Users/ronitguptaaa/Documents/Music-Fusion-APP/Staging/SpotifyMusic`")

In [32]:
artists_df = df.select(F.col('Artists')).distinct()
artists_list = [row['Artists'] for row in artists_df.collect()]

news_api_key = News_API

news_data = []
for artist in artists_list:
    url = f"https://newsapi.org/v2/everything?q={artist}&pageSize=10&apiKey={news_api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        articles = response.json()['articles']
        for article in articles:
            news_data.append({
                'Artist': artist,
                'Title': article.get('title'),
                'Description': article.get('description'),
                'URL': article.get('url'),
                'Content': article.get('content')
            })
    else:
        print(f"Failed to fetch news for artist: {artist}, Status Code: {response.status_code}")

news_df = pd.DataFrame(news_data)
news_spark_df = spark.createDataFrame(news_df)

news_spark_df.write.mode('overwrite').format('parquet').save('../Mart/Spotify-Details')

24/06/20 16:08:28 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [34]:
df = pd.read_parquet('/Users/ronitguptaaa/Documents/Music-Fusion-APP/Mart/Spotify-Details')
df = df.dropna()

In [35]:
columns = ", ".join([f"`{col}` VARCHAR(1024)" for col in df.columns])
drop_table_query = f"DROP TABLE IF EXISTS SpotifyDetails;"
cursor.execute(drop_table_query)
create_table_query = f"CREATE TABLE IF NOT EXISTS SpotifyDetails ({columns});"
cursor.execute(create_table_query)

for index, row in df.iterrows():
    values = tuple(row.values)
    insert_query = f"REPLACE INTO SpotifyDetails VALUES {values};"
    cursor.execute(insert_query)

alter_table_query = f"DELETE FROM SpotifyDetails WHERE Title = '[Removed]'"
cursor.execute(alter_table_query)

conn.commit()

Complete Artist Lists Details - Popularity/Followers

In [4]:
df = spark.sql("""
        SELECT DISTINCT Artist FROM 
        PARQUET.`/Users/ronitguptaaa/Documents/Music-Fusion-APP/Mart/MusicFusion-Details` 
        ORDER BY Artist
    """)

In [8]:
auth_header = base64.b64encode((spotify_secrets.get("client_id")+':'+spotify_secrets.get("client_secret")).encode()).decode()
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Authorization": f"Basic {auth_header}"
}
data = {
    "grant_type": "client_credentials"
}

response = requests.post(spotify_auth_url, headers=headers, data=data)
response_data = response.json()['access_token']

headers = {
    "Authorization": f"Bearer {response_data}"
}

response = requests.get(spotify_url, headers=headers)
spotify_response_data=response.json()

In [9]:
artist_names = [row['Artist'] for row in df.collect()]

artist_details = []

for artist_name in artist_names:
    spotify_api_url = f'https://api.spotify.com/v1/search?q={artist_name}&type=artist'
    headers = {
        'Authorization': f'Bearer {response_data}'
    }

    try:
        response = requests.get(spotify_api_url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            artists = data.get('artists', {}).get('items', [])

            for artist_data in artists:
                if artist_data.get('name').lower() == artist_name.lower():
                    artist_details.append({
                        'name': artist_data.get('name'),
                        'id': artist_data.get('id'),
                        'image_url': artist_data.get('images')[0].get('url') if artist_data.get('images') else None,
                        'followers': artist_data.get('followers', {}).get('total'),
                        'popularity': artist_data.get('popularity')
                    })
                    break

    except requests.exceptions.RequestException as e:
        print(f"Request failed for artist '{artist_name}': {e}")

artist_df = spark.createDataFrame(artist_details)

artist_df.write.mode('overwrite').format('parquet').save('../Mart/ArtistDetails')

24/06/20 16:17:01 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [ ]:
df = pd.read_parquet('/Users/ronitguptaaa/Documents/Music-Fusion-APP/Mart/ArtistDetails')

In [12]:
columns = ", ".join([f"`{col}` VARCHAR(1024)" for col in df.columns])
drop_table_query = f"DROP TABLE IF EXISTS ArtistDetails;"
cursor.execute(drop_table_query)
create_table_query = f"CREATE TABLE IF NOT EXISTS ArtistDetails ({columns});"
cursor.execute(create_table_query)

for index, row in df.iterrows():
    values = tuple(row.values)
    insert_query = f"REPLACE INTO ArtistDetails VALUES {values};"
    cursor.execute(insert_query)
conn.commit()

Integrate it with Apple Music and Youtube Music

In [14]:
df = spark.sql(
    """WITH AppleCTE AS (
        SELECT DISTINCT `Artist ID` AS AppleArtistID, `Artist Name` AS AppleArtist FROM 
        PARQUET.`/Users/ronitguptaaa/Documents/Music-Fusion-APP/Staging/AppleMusic`
    ),
    ArtistDetailsCTE AS (
        SELECT CAST(followers AS INT),
    id AS SpotifyArtistID,
    name AS SpotifyArtist,
    CAST(popularity AS INT) AS Popularity,
    image_url FROM PARQUET.`/Users/ronitguptaaa/Documents/Music-Fusion-APP/Mart/ArtistDetails`
    ),
    YoutubeMusicCTE AS (
        SELECT DISTINCT 
        `Artist Name` AS YoutubeArtist,
        `Artist IDs` AS YoutubeArtistID 
        FROM PARQUET.`/Users/ronitguptaaa/Documents/Music-Fusion-APP/Staging/YoutubeMusic`
    )
    SELECT *
    FROM ArtistDetailsCTE AS AD 
    INNER JOIN AppleCTE AS ACTE 
    ON LTRIM(RTRIM(LOWER(AD.SpotifyArtist))) = LTRIM(RTRIM(LOWER(ACTE.AppleArtist)))
    LEFT JOIN YoutubeMusicCTE AS YM 
    ON LTRIM(RTRIM(LOWER(AD.SpotifyArtist))) = LTRIM(RTRIM(LOWER(YM.YoutubeArtist)))
    """
)

df.write.mode('overwrite').format('parquet').save('../Mart/ArtistDetailsFinal')

24/06/20 16:21:22 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [ ]:
data = pd.read_parquet('/Users/ronitguptaaa/Documents/Music-Fusion-APP/Mart/ArtistDetailsFinal')

In [16]:
drop_table_query = "DROP TABLE IF EXISTS ArtistDetailsFinal;"
cursor.execute(drop_table_query)

columns = ", ".join([f"`{col}` INT" if col == 'popularity' else f"`{col}` VARCHAR(1024)" for col in data.columns])
create_table_query = f"CREATE TABLE IF NOT EXISTS ArtistDetailsFinal ({columns});"
cursor.execute(create_table_query)

insert_query = f"INSERT INTO ArtistDetailsFinal ({', '.join([f'`{col}`' for col in data.columns])}) VALUES ({', '.join(['%s' for _ in data.columns])})"

for index, row in data.iterrows():
    values = tuple(None if pd.isnull(value) else value for value in row.values)
    cursor.execute(insert_query, values)
    
conn.commit()